In [ ]:
# importing packages

import pandas as pd
import numpy as np

from hashlib import sha256
import json

import matplotlib.pyplot as plt


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Background

Depois de meses a fazer, candidatures finalmente consegues um estágio de Verão no laboratório **LabOfLife (LOL)**

Começas o estágio. És inserido numa equipa de laboratório que analisa proteínas e as suas propriedades. Nos primeiros dias és introduzido aos diversos projetos que o laboratório tem e também à equipa onde te vais inserir, e mal podes esperar por começar a aprender e a aplicar os teus conhecimentos.

Certo dia chegas ao laboratório bem cedo, e mal entras na sala ficas surpreso por ver toda a tua equipa já lá, e ainda por cima o ambiente parece pesado... Preocupado, chegas-te a uma colega e perguntas o que se está a passar. Ela diz-te: 


> "Estamos a analisar as fosforilações presentes num proteoma de alfa-casína. Para isso fizémos uma experiência e recolhemos os dados todos para o ficheiro `data/dados.csv`. No entanto passámos a noite toda a tentar terminar a nossa análise das fosforilações, mas a única coisa que conseguimos fazer foi uma função que está no ficheiro `utils.py` que extrai só os eventos de fosforilações da coluna que contém todas as modificações que aconteceram às proteínas"

Aprontamente, tu ofereces a tua ajuda:

> "Não te preocupes. Eu fiz um workshop de Pandas e Python organizado pelo ANEEB e consigo-vos ajudar, explica-me melhor a situação e eu resolvo isto em menos de 1h."

Os teus colegas agradecem-te, explicam a situação de forma detalhada e vão descansar para casa.


**Problema**:
> Estamos a testar umas nanopartículas (IMAC – immobilized ion metal afinity chromatography) que servem para pré-concentrar os fosfopeptidos. Queremos testar se a nossa concentração foi bem feita olhando para as fosforilações das sequencias de aminoácidos.

**Objetivo**:

> Analisar o número total de fosforilações por sequência de aminoácidos, bem como o número de fosforilações que ocorrem nos aminoácidos S, T e Y. Queremos verificar se rondamos a distribuição esperada: 80% de S, 18% de T e 2% de Y. Caso os resultados apontem para números muito diferentes estamos perante selectividades diferentes.


**Dados**:

> Os dados com que vais trabalhar resultam de análise por espectrometria de massa da pré-concentração usando IMACs de titânio, európio e samário por um programa chamado MASCOT. O dataset contém muita informação, mas para esta análise só nos interessa olhar para a **sequência** de aminoácidos e as fosforilações que estão presentes na coluna de **modificações**.

# Exercise 1 - Prepare de data

## Exercise 1.1 - Read the data

The url for the data is the following : `<inser url here>`

In [ ]:
# df = ... # uncomment to provide an answer

# df.head() # uncomment to inspect the data

Evaluation cell below

In [ ]:
# don't forget to hide contents by default
assert df.shape==(2541,20), "Oops!"

## Exercise 1.2 - Select relevant columns

There's a lot of information on this dataset, but you remember that the problem consists of analyzing only the phosphorylation events out of all **Modifications** that occurred on the **Sequence** of aminoacids. Look for the 2 columns that contain this information and create a new dataframe that only contains these columns.

In [ ]:
#relevant_columns = [... , ...] # uncomment to provide an answer. hint: this should be a list of two strings

#phospho_df = df[relevant_columns] # uncomment to provide an answer
#phospho_df.head() 

In [ ]:
hash_df = b'\xe9\x9c\xc8\x8d9\xc9\x9e\x1d\xf2\xe4B\xc6\xc1S+\x9aPt\xa8\x89\xc8\x00[!\xd8:-\xfa~\xb0\xae\xcd'

assert phospho_df.shape==(2541,2), "Oops!"
assert sha256(phospho_df.reindex(sorted(phospho_df.columns), axis=1).to_dict().__str__().encode()).digest() == hash_df

# Exercise 2

## Exercise 2.1 - Extract the aminoacid sequence

We have the data we need. The `Modifications` column contains the position of each reaction in the sequence. However, the `Sequence` column needs some cleaning because each aminoacid sequence is in the middle of two full stops: "\<Letter\>.**\<sequence\>**.\<Letter\>"

**Objective**: create a function that receives a sequence in its raw form and extracts only the actual sequence.

In [ ]:
def extract_sequence(sequence):
    """
    Extracts the actual aminoacid sequence from a sequence produced by MASCOT.
    
    Parameters
    ----------
    
    sequence : str
        An sequence produced by the program MASCOT.
        
    Returns
    -------
    
    amino_sequence : str
        The aminoacid sequence
    
    Examples
    --------
    
    sequence = 'K.VPQLEIVPNSAEER.L'
    extract_sequence(sequence)
    >>> 'VPQLEIVPNSAEER'
    
    """
    
    # amino_sequence = ...
    
    return amino_sequence

# try it yourself!
# example = 'K.VPQLEIVPNSAEER.L' # uncomment to test the function
# extract_sequence(example) # uncomment to test the function

In [ ]:
test_example_1 = 'T.VPQLEIVPNSAEER.K'
test_example_2 = 'U.VGQLAIVPSSAEUDR.S'
test_example_3 = 'P.SDQLGIVPRSAEER.U'

assert extract_sequence(test_example_1) == 'VPQLEIVPNSAEER'
assert extract_sequence(test_example_2) == 'VGQLAIVPSSAEUDR'
assert extract_sequence(test_example_3) == 'SDQLGIVPRSAEER'

# Exercise 2.2 - Clean the Sequence column

In [ ]:
# we are making a copy so we don't change the original data
phospho_cleaned_df= phospho_df.copy()

Now that you have a function that extracts a sequence, use that function to clean the entire `Sequence` column of the dataframe.

*hint*: you can use the function you created# we are making a copy so we don't change the original data
phospho_cleaned_df= phospho_df.copy() to help you here

In [ ]:
# Answer below

#df_cleaned['Sequence'] = ...

# df_cleaned.head(5) # uncomment to test

In [ ]:
hash_correct_answer = b'\xaf\x00\x93\xb8<}\x9f\xe6\xe3\xfd2O\xf8\xa9TW(\xb1\x08\xf1\xdc\xdc|\xc8\xc8\xb5\xc6qD~*G'
assert sha256(phospho_cleaned_df.Sequence.__str__().encode()).digest() == hash_correct_answer

# Exercise 3 - Extract aminoacids with phosphorylation events


We know that only the aminoacids `S`, `T`, and `Y` contain phosphorylation events. We want to analyse the distribution of these events to compare with the theoretical expected distribution: 80% of S, 18% of T, and 2% of Y.


Now is the perfect time to use the function that your lab colleagues created: `capture_phospho_positions`. You don't need to understand the code inside this function. All you need to know is that this function receives a string of modifications and outputs the positions of phospho events in a list

In [ ]:
def capture_phospho_positions(modification):
    """
    Given a modification description of the form:
    "Acetyl: 1; Oxidation: 4; Phospho: 6", captures the positions of the Phospho component only
    
    
    Returns a list of the positions.
    
    Parameters
    ----------
    modification: str
        String containing the modifications to a sequence of aminoacids.
        
    Returns
    -------
    positions_list : list
        List of all the aminoacids with Phospho events
    
    
    Examples
    --------
    
    example = "Acetyl: 1; Oxidation: 4; Phospho: 6"
    capture_phospho_positions(example)
    >>> [6]
    
    example_2 = "Acetyl: 5; Oxidation: 2; Phospho: 6, 9"
    capture_phospho_positions(example)
    >>> [6, 9]
    """
    
    
    events = modification.split(';')
    phospho_info = [s for s in events if 'Phospho' in s][0]
    
    # remove unecessary info ('Phospho:')
    positions_str = phospho_info[phospho_info.find(':')+1:]
    # remove whitespaces and get list of positions
    positions_list = positions_str.replace(" ", "").split(',')
    # convert positions to ints
    positions_list = [int(e) for e in positions_list]
    
    return positions_list

## Exercise 3.1 - Extract the aminoacids with phospho events

In [ ]:
def get_phospho_aminoacids(row):
    """
    Given a pandas series with two elements: Sequence and Modifications,
    returns a list with all the aminoacids where a Phospho event occured.
    
    
    Parameters
    ----------
    row : pd.Series, index = ['Sequence', 'Modifications']
        A sequence and the respective modifications.
        
        
    Returns
    -------
    phospho_amino_list : list
        A list of all the aminoacids with phospho events
    
    Examples
    --------
    modification = "Acetyl: 1; Oxidation: 4; Phospho: 6, 11"
    sequence_cleaned = 'TVDMESTEVFTK'
    row = pd.Series([sequence, modification], index=['Sequence', 'Modifications'])
    
    get_phospho_aminoacids(row)
    >>> ["S",""]
    """
    
    
    return phospho_amino_list

# Try it yourself!
# modification = "Acetyl: 1; Oxidation: 4; Phospho: 6, 11"
# sequence_cleaned = 'TVDMESTEVFTK'
# row = pd.Series([sequence, modification], index=['Sequence', 'Modifications'])
# get_phospho_aminoacids(row) # uncomment to test your function

In [ ]:
sequence_cleaned = 'TVDMESTEVFTK'

test_case_1 = pd.Series([sequence_cleaned, "Acetyl: 4; Phospho: 6, 11"], index=['Sequence', 'Modifications'])
test_case_2 = pd.Series([sequence_cleaned, "Oxidation: 2; Phospho: 2, 4, 5"], index=['Sequence', 'Modifications'])
test_case_3 = pd.Series([sequence_cleaned, "Phospho: 3"], index=['Sequence', 'Modifications'])

assert get_phospho_aminoacids(test_case_1) == ["S","T"]
assert get_phospho_aminoacids(test_case_2) == ["V","M","E"]
assert get_phospho_aminoacids(test_case_3) == ["D"]

## Exercise 3.2 - Create a new column with the aminoacids' list

Now that you can extract the specific aminoacids with phospho events, let's add that information to a new column!

In [ ]:
# phospho_cleaned_df['phospho_amino_list'] = ...

# phospho_cleaned_df.head(5) # uncomment to look at the outputs

In [ ]:
hash_correct_answer = b'8\x12\xe1W\xbd\x10\x93\xcbc\x98:L\x0f\xf1T\xb3l\x1b\xf6\xe1\x16F\xd6K\x992\xc8\x0f\xe4\xf3\xd1\x96'
assert sha256(phospho_cleaned_df.phospho_amino_list.__str__().encode()).digest() == hash_correct_answer

# Exercise 3.3 - Count aminoacids

Now that the aminoacids are in a list, we just need to count them!

**Objective**: Create a function that receives: an aminoacid letter, and an aminoacid list and counts the number of times that letter appears in the list

*hint: a lot of the times googling specific questions can really help you because someone may have already asked the same thing. If you ask the right question you may be able to solve this exercise with one line*

In [ ]:
def count_occurrence(letter,amino_list):
    """
    given a specific aminoacid and a list of aminoacids,
    counts the number of times that aminoacids appears in the list.
    
    
    Parameters
    ----------
    
    letter : str
        An aminoacid letter
    
    amino_list : list
        A list of aminoacid letters
    
    Returns
    -------
    
    count : int
        The number of occurrences of that letter in the list
    
    
    Example
    -------
    letter = 'T'
    amino_list = ['S','Y','S','T']
    count_occurrence(letter, amino_list)
    >>> 1
    amino_list_2 = ['T','Y','S','T']
    count_occurrence(letter, amino_list_2)
    >>> 2
    """
    
    # count = 
    
    return count

# try it yourself!
# letter = 'K'
# amino_list = ['A','K']
# count_occurrence(letter,amino_list) # uncomment to test the function

In [ ]:
test_case_1 = {'letter':'T', 'amino_list':['S','Y','S','T']}
test_case_2 = {'letter':'Y', 'amino_list':['Y','Y']}
test_case_3 = {'letter':'S', 'amino_list':['S','S','S','T','S']}

assert count_occurrence(**test_case_1) == 1
assert count_occurrence(**test_case_2) == 2
assert count_occurrence(**test_case_3) == 4

## Exercise 3.4 - Create a column with the occurences of the aminoacid "S"

We are almost ready to analyze the distribution of the aminoacids with phospho events!

Let's use the previous function to generate a new column that counts the occurrences of phospho events in S for each row

In [ ]:
# phospho_cleaned_df['S'] = ...

# phospho_cleaned_df.head(5) # uncomment to test the output

In [ ]:
hash_correct_answer = b'\x0eq\x1d\t`\xa6PM\x9e\xc1\x8d\x1a#\xea\x97\xe2L\xfa\x86H\x1b\x1e}\x95kO\xd0\xa6\x01.\x1b)'
assert sha256(phospho_cleaned_df['S'].__str__().encode()).digest() == hash_correct_answer

## Exercise 3.5 - Let's now do the same for the aminoacids T and Y

In [ ]:
# phospho_cleaned_df['T'] = ...
# phospho_cleaned_df['Y'] = ...

# phospho_cleaned_df.head(5) # uncomment to test the output

In [ ]:
hash_correct_answer_T = b"\xcc\xa56z5j|mJ:)\x8d\\\n0\x80\xe5!'\xf2i\xf0\xe3L\xc6\xdd\xdb\xe13i#\x91"
hash_correct_answer_Y = b'Z\xa0V\xc0\xc1\x99\xc0z\x1b\xd3\t.v\x9d2a\x19\xe1\x8c?\x91\xe3\xcd\xe6={Dl\x9d\x8apt'

assert sha256(phospho_cleaned_df['T'].__str__().encode()).digest() == hash_correct_answer_T
assert sha256(phospho_cleaned_df['Y'].__str__().encode()).digest() == hash_correct_answer_Y

# Exercise 4 - Analyzing the aminoacids

Congrats for reaching here! We are almost done with our analysis, since we have the data ready. 

Our original goal was to compare the experimental distribution of the aminoacids S,T and Y with the theoretical one.  Let's do that then!

**Objective**: create a function that receives the whole dataframe and prints the percentage of total events of S, T and Y



In [ ]:
def report_amino_distribution(dataframe):
    """
    Given a dataframe with S,T and Y columns that correpond to the respective count
    of phospho events in these aminoacids, report their distribution in the dataset.
    """
    
    
    # S_total_count = 
    # T_total_count =
    # Y_total_count =
    
    # total = 
    
    # S_percentage = 
    # T_percentage = 
    # Y_percentage = 
    
    print(f'S: {S_percentage.round(2)}%\nT: {T_percentage.round(2)}%\nY: {Y_percentage.round(2)}%')

In [ ]:
report_amino_distribution(phospho_cleaned_df)

# Conclusions

- Despite "Y" being close to the expected distribution, phospho events in the "S" aminoacid happen a lot more than they should (96% vs 80%), and at the same time the "T" aminoacids are below what is expected (1.8% vs 18%)

# That's it!

<img src="https://64.media.tumblr.com/e68ad3308e9701c10ca6aebe56008be5/tumblr_inline_p6nhl0n3qv1vvskl5_640.jpg">